In [1]:
import os
import random
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.transformation import *
from utils.models import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.get_logger().setLevel(logging.ERROR)

catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")
mf_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")

2024-10-27 06:09:15.205990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-27 06:09:15.216032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-27 06:09:15.228971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-27 06:09:15.229026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 06:09:15.237916: I tensorflow/core/platform/cpu_feature_gua

In [2]:
catalogo = calculate_ratio_of_interactions(mf_dataset, catalogo)

# Parámetros para el cálculo del puntaje
features = {"hito": (1, 4), "skill": (0, 15), "knowledge": (1, 15), "interaction_ratio": (0.0, 1.0)}
weights = {"hito": 0.3, "skill": 0.25, "knowledge": 0.25, "interaction_ratio": 0.2}

# Calcular el puntaje en el DataFrame
catalogo = calculate_score_dataset(catalogo, features, weights, new_column='formula')

scale_complexity_norm = MinMaxScaler()
catalogo['complexity_norm'] = scale_complexity_norm.fit_transform(catalogo[['complexity']])

scale_complexity12_norm = MinMaxScaler()
catalogo['complexity12_norm'] = scale_complexity12_norm.fit_transform(catalogo[['complexity12']])

df_catalogo = catalogo.copy()
df_catalogo.head()

,oid,nombre,h1,h2,h3,h4,s1,s2,s3,s4,...,k4,hito,skill,knowledge,complexity,complexity12,interaction_ratio,formula,complexity_norm,complexity12_norm
0,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,1,...,1,1,1,7,23,279,0.787206,0.281251,0.111111,0.022774
1,16f619db31204ded9418136c4587ddd8,Calculadora GeomÃ©trica,0,0,1,0,0,0,0,1,...,1,2,1,1,17,529,0.359008,0.188468,0.080808,0.281573
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,...,1,2,3,7,55,567,0.022193,0.261581,0.272727,0.320911
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,0,0,0,1,0,0,0,0,...,1,1,0,3,3,259,0.707572,0.177229,0.010101,0.002070
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,...,1,1,0,1,1,257,0.956919,0.191384,0.000000,0.000000


In [3]:
df_ratings = factorization_to_ratings(df_catalogo, mf_dataset, user_col='rut')
train_data, test_data = train_test_split(df_ratings, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
train_data.head(10)

,rut,oid,nombre,h1,h2,h3,h4,s1,s2,s3,...,k4,hito,skill,knowledge,complexity,complexity12,interaction_ratio,formula,complexity_norm,complexity12_norm
1197,89,52620b0c858a4c59bc324b65278d28bd,Cajero AutomÃ¡tico Nivel 2,0,0,0,1,0,0,0,...,1,1,1,3,19,275,0.304178,0.113216,0.090909,0.018634
8052,1099,7f60644b0a1b484681ae5c8e36166c58,Signo del ZodÃ­aco,0,0,0,1,0,0,0,...,1,1,1,1,17,273,0.848564,0.186379,0.080808,0.016563
5461,403,0973dae0e1b74ab8baa8d94339ee3ae6,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,...,1,1,1,7,23,279,0.787206,0.281251,0.111111,0.022774
7389,1026,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.050914,0.000000,0.000000
6478,923,46850a246d48484b8f104f8aab5679b6,Descomponer un nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.781984,0.192111,0.010101,0.002070
1429,104,718578451f3f4eca87437cadfe98d688,Suma de los N primeros nÃºmeros naturales,0,0,0,1,0,0,0,...,0,1,0,2,2,258,0.962141,0.210285,0.005051,0.001035
4464,336,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.050914,0.000000,0.000000
4092,307,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.956919,0.191384,0.000000,0.000000
2072,153,ac7382763e484d37908da54c076f7577,Cajero AutomÃ¡tico Nivel 1,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.515666,0.138847,0.010101,0.002070
8660,1159,729d37da8f2d46f3af2d891df04949ef,Juego Adivina mi nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.507833,0.137281,0.010101,0.002070


In [5]:
Y_TRAIN = train_data['rut'].values
Y_TEST = test_data['rut'].values


## COMLEXITY (BINARY)
x_train_complexity_binary = train_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'],axis=1).values
x_test_complexity_binary = test_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'],axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_binary = train_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_binary = test_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## FORMULA (BINARY)
x_train_formula_binary = train_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_formula_binary = test_data.drop(labels=['rut', 'nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values


## COMLEXITY (LABEL ENCODED)
x_train_complexity_label = train_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity_label = test_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_label = train_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_label = test_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values
## FORMULA (BINARY)
x_train_formula_label = train_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_formula_label = test_data.drop(labels=['rut', 'nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values

In [6]:
learning_rate = 0.001
epochs = 50
batch_size = 32
validation_split = 0.2
dropout_rate = 0.2
layer_units = [64, 32]

# Binary models

In [7]:
model_complexity_binary = TargetModel(input_shape=x_train_complexity_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_binary = model_complexity_binary.train(x_train_complexity_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_binary = model_complexity_binary.evaluate(x_test_complexity_binary, Y_TEST)
predictions_complexity_binary = model_complexity_binary.predict(x_test_complexity_binary)

2024-10-27 06:10:14.800643: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.834371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.834409: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.835821: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.835858: I external/local_xla/xla/stream_executor

Epoch 1/50


/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.944150: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.944207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-27 06:10:14.944213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2019] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-10-27 06:10:14.944237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000

149/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 495559.8750

I0000 00:00:1730020218.066717  165535 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 488485.8438 - val_loss: 365431.4375
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 360516.8438 - val_loss: 277919.5938
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 273547.9062 - val_loss: 193613.8281
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 199262.8125 - val_loss: 182297.1875
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 193530.5312 - val_loss: 181977.5781
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 199415.0938 - val_loss: 180896.7500
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 195232.1406 - val_loss: 180746.9844
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 193711.7969 - val_loss: 180021.7812
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 191456.8750 - val_loss: 179483.2812
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 193578.7031 - val_loss: 179632.4219
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

In [8]:
model_complexity12_binary = TargetModel(input_shape=x_train_complexity12_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_binary = model_complexity12_binary.train(x_train_complexity12_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_binary = model_complexity12_binary.evaluate(x_test_complexity12_binary, Y_TEST)
predictions_complexity12_binary = model_complexity12_binary.predict(x_test_complexity12_binary)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 350814.8438 - val_loss: 256884.8906
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 270720.7500 - val_loss: 255678.2969
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 268851.9375 - val_loss: 255055.9375
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 264706.1250 - val_loss: 252318.2031
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 263957.2500 - val_loss: 250307.9844
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 259508.0938 - val_loss: 247313.3906
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 259476.6406 - val_loss: 244212.1094
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 254381.5000 - val_loss: 238868.8125
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 250493.3750 - val_loss: 230610.1719
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 232294.7656 - val_loss: 215098.0781
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━

In [9]:
model_formula_binary = TargetModel(input_shape=x_train_formula_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_formula_binary = model_formula_binary.train(x_train_formula_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_formula_binary = model_formula_binary.evaluate(x_test_formula_binary, Y_TEST)
predictions_formula_binary = model_formula_binary.predict(x_test_formula_binary)


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 509641.9688 - val_loss: 334642.5312
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 263788.4375 - val_loss: 188428.0938
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 198317.1875 - val_loss: 181426.7812
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 193790.0156 - val_loss: 179580.4062
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 190895.2188 - val_loss: 179119.4219
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 189900.0312 - val_loss: 178225.7188
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 186429.9688 - val_loss: 177889.7656
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 186186.3594 - val_loss: 177373.7031
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 187842.9375 - val_loss: 177091.4844
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 187044.5938 - val_loss: 176837.9062
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━

# Labels models

In [10]:
model_complexity_label = TargetModel(input_shape=x_train_complexity_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_label = model_complexity_label.train(x_train_complexity_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_label = model_complexity_label.evaluate(x_test_complexity_label, Y_TEST)
predictions_complexity_label = model_complexity_label.predict(x_test_complexity_label)


Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 493067.1250 - val_loss: 363230.1875
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 378764.7188 - val_loss: 303064.5000
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 308040.5625 - val_loss: 229596.2812
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 242991.4844 - val_loss: 206536.4531
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 219977.2344 - val_loss: 203871.9531
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 213822.6094 - val_loss: 202948.1875
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 210351.1094 - val_loss: 201919.7812
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 208088.2969 - val_loss: 200482.6875
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 210578.7812 - val_loss: 199025.3125
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 211113.0781 - val_loss: 198248.8438
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━

In [11]:
model_complexity12_label = TargetModel(input_shape=x_train_complexity12_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_label = model_complexity12_label.train(x_train_complexity12_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_label = model_complexity12_label.evaluate(x_test_complexity12_label, Y_TEST)
predictions_complexity12_label = model_complexity12_label.predict(x_test_complexity12_label)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 354676.4375 - val_loss: 259625.8281
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 278954.7188 - val_loss: 257933.5781
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 269383.7188 - val_loss: 258369.3906
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 279784.9688 - val_loss: 256048.9844
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 265705.5000 - val_loss: 256041.1719
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 271923.0312 - val_loss: 255092.1875
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 269139.0625 - val_loss: 253942.0000
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 263967.1875 - val_loss: 252925.0625
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 263258.9062 - val_loss: 251890.7656
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 264712.4375 - val_loss: 250732.7344
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━

In [12]:
model_formula_label = TargetModel(input_shape=x_train_formula_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_formula_label = model_formula_label.train(x_train_formula_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_formula_label = model_formula_label.evaluate(x_test_formula_label, Y_TEST)
predictions_formula_label = model_formula_label.predict(x_test_formula_label)

Epoch 1/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 480508.3750 - val_loss: 340595.7188
Epoch 2/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 314754.0938 - val_loss: 245514.8125
Epoch 3/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 259401.6562 - val_loss: 229451.2812
Epoch 4/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 243837.3281 - val_loss: 215442.3906
Epoch 5/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 228963.0156 - val_loss: 204202.3594
Epoch 6/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212999.0469 - val_loss: 197512.6562
Epoch 7/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 214391.7812 - val_loss: 194753.7812
Epoch 8/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 205499.4688 - val_loss: 193971.8125
Epoch 9/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 203494.1875 - val_loss: 193085.3906
Epoch 10/50
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197395.7656 - val_loss: 192468.1250
Epoch 11/50
202/202 ━━━━━━━━━━━━━━━━━━

# Results

### Estudiante

In [13]:
# 0 | 1305
rut_estudiante = 0
ejercicios_realizados = df_ratings[df_ratings['rut'] == rut_estudiante]['oid'].values
ejercicios_no_realizados = catalogo[~catalogo['oid'].isin(ejercicios_realizados)]
catalogo[catalogo['oid'].isin(ejercicios_realizados)].sort_values(by='complexity12', ascending=True)[['oid', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,oid,nombre,hito,skill,knowledge,complexity
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,1,0,1,1
25,89f44e7f5842479fb283e43c52ce067b,Ordenar tres nÃºmeros,1,0,1,1
17,718578451f3f4eca87437cadfe98d688,Suma de los N primeros nÃºmeros naturales,1,0,2,2
23,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,1,0,2,2
3,171b5e86d4fb47268f2692587fbec073,NÃºmeros Primos,1,0,3,3
10,46850a246d48484b8f104f8aab5679b6,Descomponer un nÃºmero,1,0,3,3
18,729d37da8f2d46f3af2d891df04949ef,Juego Adivina mi nÃºmero,1,0,3,3
26,8f24397e36034cccb71e9d578975c33d,Contestador AutomÃ¡tico,1,0,3,3
29,a3963220090f4e50a266ce53d33b9841,AprobaciÃ³n de CrÃ©ditos,1,0,3,3
42,d8395f43e4a1454d90346ac5a1ba561a,NÃºmeros Amigos,1,0,3,3


### Resultados

In [15]:
predicciones_complexity_binary = model_complexity_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_binary = model_complexity12_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_formula_binary = model_formula_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


predicciones_complexity_label = model_complexity_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_label = model_complexity12_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'interaction_ratio', 'formula', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_formula_label = model_formula_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'oid', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'interaction_ratio', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


In [16]:
recomendations_complexity_binary = ejercicios_no_realizados.copy()
recomendations_complexity_binary['predicted_preference'] = predicciones_complexity_binary
recomendations_complexity_binary = recomendations_complexity_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['oid'].isin(recomendations_complexity_binary['oid'].values)][['oid', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,oid,nombre,hito,skill,knowledge,complexity
5,24b87304a1eb4e95856da7f574a66fe8,Validador de Expresiones MatemÃ¡ticas,4,8,7,135
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,1
13,52620b0c858a4c59bc324b65278d28bd,Cajero AutomÃ¡tico Nivel 2,1,1,3,19
30,abbcd12771fd45ca823f6222f72b48c8,Interpretar Expresiones MatÃ©maticas,4,12,7,199
32,ac7382763e484d37908da54c076f7577,Cajero AutomÃ¡tico Nivel 1,1,0,3,3
35,baf2f8e0167a4e089d2cec16582c9ae9,Factores Primos,1,1,3,19
36,be17b556785c428e8dabbef0c9d11f6f,Pintar RegiÃ³n,4,8,7,135
46,dd1000315e5b43ac89ae1d416400d9d6,Pokemon,3,3,15,63
47,e6093c934b8c40b4bee9969b32a30bed,El problema de la mochila,4,12,7,199
51,eb91ed4e997b4ca98f3c86e3e86a3755,Combinaciones de NÃºmeros Binarios,4,12,7,199


In [17]:
recomendations_complexity12_binary = ejercicios_no_realizados.copy()
recomendations_complexity12_binary['predicted_preference'] = predicciones_complexity12_binary
recomendations_complexity12_binary = recomendations_complexity12_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['oid'].isin(recomendations_complexity12_binary['oid'].values)][['oid', 'nombre', 'hito', 'skill', 'knowledge', 'complexity12']]

,oid,nombre,hito,skill,knowledge,complexity12
5,24b87304a1eb4e95856da7f574a66fe8,Validador de Expresiones MatemÃ¡ticas,4,8,7,1159
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,257
13,52620b0c858a4c59bc324b65278d28bd,Cajero AutomÃ¡tico Nivel 2,1,1,3,275
30,abbcd12771fd45ca823f6222f72b48c8,Interpretar Expresiones MatÃ©maticas,4,12,7,1223
32,ac7382763e484d37908da54c076f7577,Cajero AutomÃ¡tico Nivel 1,1,0,3,259
35,baf2f8e0167a4e089d2cec16582c9ae9,Factores Primos,1,1,3,275
36,be17b556785c428e8dabbef0c9d11f6f,Pintar RegiÃ³n,4,8,7,1159
47,e6093c934b8c40b4bee9969b32a30bed,El problema de la mochila,4,12,7,1223
50,e9df502fccd9455a8a7609d542a857c8,MCM recursivo,4,1,3,1043
51,eb91ed4e997b4ca98f3c86e3e86a3755,Combinaciones de NÃºmeros Binarios,4,12,7,1223


In [18]:
recomendations_formula_binary = ejercicios_no_realizados.copy()
recomendations_formula_binary['predicted_preference'] = predicciones_formula_binary
recomendations_formula_binary = recomendations_formula_binary.sort_values(by='predicted_preference', ascending=False).head(10)
catalogo[catalogo['oid'].isin(recomendations_formula_binary['oid'].values)][['oid', 'nombre', 'hito', 'skill', 'knowledge', 'formula']]

,oid,nombre,hito,skill,knowledge,formula
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,1,0,1,0.050914
16,677fbf669a0d496c843190c116b4f173,OrnitologÃ­a 1,3,0,8,0.325000
24,82e092a2744544d1b6cedde23d4596a5,Clase Vector3D,3,1,15,0.466667
34,b5e026a9cf014d71b110c58108a21752,OrnitologÃ­a 3,3,0,12,0.396429
41,d44d5fead16c47a986287ff2bc48d20f,Clase FechaHora,3,2,15,0.483333
43,d849ec19c3834c2b9c9ff2ab34d2f3e3,Clase Usuario,3,3,15,0.500000
45,db873d7e50684080ba8f39aed56b288e,Carro de Compras,3,2,15,0.483333
46,dd1000315e5b43ac89ae1d416400d9d6,Pokemon,3,3,15,0.500000
49,e8a9dc7b6127482987d9b6dbca73ec5e,Caminos entre ciudades,3,2,15,0.483333
52,f7f404137f974160bcbfa423f5d35eda,OrnitologÃ­a 2,3,0,8,0.325000
